In [ ]:
# import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from notebook.services.config import ConfigManager
import logging
import requests
import base64
import concurrent.futures
from ratelimit import limits, sleep_and_retry

In [ ]:
# reload the data
streams_df = pd.read_csv(r'/Users/adityamxr/Desktop/spotify-time-series/data-fetching/streams_df_1.csv')

In [ ]:
# increase the IOPub data rate limit to prevent the notebook from stopping output 
# when processing large volumes of data or handling frequent API rate limiting messages

cm = ConfigManager().update('notebook', {'NotebookApp': {'iopub_data_rate_limit': 100000000}})

In [ ]:
# set up logging
logging.basicConfig(level=logging.INFO)

# spotify API credentials
client_id = '56b1d34cf1574e28855dc07f73f7754c'
client_secret = '490ebd868e76461fbd7dcb7a0a9cce8f'

# cache to store artist-genre mappings
genre_cache = {}

# function to get the access token
def get_access_token(client_id, client_secret):
    token_url = "https://accounts.spotify.com/api/token"
    credentials = f"{client_id}:{client_secret}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()

    headers = {
        "Authorization": f"Basic {encoded_credentials}"
    }
    data = {
        "grant_type": "client_credentials"
    }

    response = requests.post(token_url, headers=headers, data=data)
    response.raise_for_status()
    access_token = response.json().get('access_token')
    return access_token

In [ ]:
# get the access token
access_token = get_access_token(client_id, client_secret)

# rate-limited function to fetch genres for a given artist
@sleep_and_retry
@limits(calls=10, period=1)
def get_artist_genres(artist_name, access_token):
    search_url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1
    }

    try:
        response = requests.get(search_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        artists = data.get('artists', {}).get('items', [])

        if not artists:
            logging.warning(f"No artist found for {artist_name}")
            return []

        artist_info = artists[0]
        genres = artist_info.get('genres', [])
        return genres

    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data for {artist_name}: {e}")
        return []


In [ ]:
# function that uses caching to fetch genres
def get_artist_genres_cached(artist_name, access_token):
    if artist_name in genre_cache:
        logging.info(f"Cache hit for '{artist_name}'")
        return genre_cache[artist_name]
    
    genres = get_artist_genres(artist_name, access_token)
    genre_cache[artist_name] = genres  # Store the result in the cache
    logging.info(f"Fetched and cached genres for '{artist_name}': {genres}")
    return genres

In [ ]:
# function to fetch genres for a list of artists in parallel, using caching and rate limiting
def fetch_genres_for_artists_parallel(df, access_token):
    # Ensure the 'genres' column exists
    df['genres'] = None

    def get_genres_for_multiple_artists(artists_list):
        all_genres = set()
        for artist in artists_list:
            if isinstance(artist, str):  # Ensure that artist is a string
                genres = get_artist_genres_cached(artist, access_token)
                all_genres.update(genres)
        return list(all_genres) if all_genres else []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(get_genres_for_multiple_artists, row['artists']): idx for idx, row in df.iterrows()}
        for future in concurrent.futures.as_completed(futures):
            idx = futures[future]
            try:
                genres = future.result()
                df.at[idx, 'genres'] = genres
                logging.info(f"Updated genres for index {idx}: {genres}")
            except Exception as e:
                logging.error(f"Error processing index {idx}: {e}")
                df.at[idx, 'genres'] = []
    return df

In [ ]:
# make sure each entry in the 'artists' column is a list
streams_df['artists'] = streams_df['artists'].apply(lambda x: x if isinstance(x, list) else [x])

# test with a small subset of the DataFrame
test_df_with_genres = fetch_genres_for_artists_parallel(streams_df, access_token)
print(test_df_with_genres.head())

In [ ]:
# Count the number of rows where the genres column is empty
empty_genre_count = streams_df['genres'].apply(lambda x: len(x) == 0).sum()

print(f"Number of rows with empty genres: {empty_genre_count}")

In [ ]:
# impute the empty lists in the genre column with "unknown"

for i in range(len(streams_df)):
    if len(streams_df.at[i, 'genres']) == 0:
        streams_df.at[i, 'genres'] = ["Unknown"]

In [ ]:
# count the empty lists again
empty_genre_count = streams_df['genres'].apply(lambda x: len(x) == 0).sum()

print(f"Number of rows with empty genres: {empty_genre_count}")

# empty lists have been imputated with "Unknown"

In [ ]:
# verify imputation
print(streams_df['genres'].head())

In [ ]:
# In depth EDA
# check shape of dataframe
print(f"Dataset Shape: {streams_df.shape}")

In [ ]:
streams_df.popularity.isnull().sum()

In [ ]:
# Calculate the range of the popularity feature
popularity_min = streams_df['popularity'].min()
popularity_max = streams_df['popularity'].max()

print(f"The range of popularity values is from {popularity_min} to {popularity_max}.")

In [ ]:
# before proceeding with EDA, let's save the dataframe to a csv since the API genre fetch took nearly 3 hours!
streams_df.to_csv('streams_df_with_genres.csv', index=False)

In [ ]:
# popularity does not have any missing values. The ID, name, artists, and explicit columns are of no 
# significance to this analysis, so we'll be dropping them

columns_drop = ['id', 'explicit','artists','name']
streams_df_clean = streams_df.drop(columns=columns_drop)
streams_df_clean.head()

In [ ]:
# no nulls exist across all features. Compute the data range for the year feature

streams_df_clean.year.min(), streams_df_clean.year.max(), streams_df_clean.year.max() - streams_df_clean.year.min()

# 99 years of data exists, so the granularity of the time series can be in years. day/month granularity is 
# unnecessary since our goal is to forecast genre popularity over the next few decades
# therefore the release date column can be dropped

In [ ]:
streams_df_clean_2 = streams_df_clean.drop(columns = 'release_date')
streams_df_clean_2.head()